In [9]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1748114708154, experiment_id='2', last_update_time=1748114708154, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747592152260, experiment_id='1', last_update_time=1747592152260, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747591349311, experiment_id='0', last_update_time=1747591349311, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment("my-cool-experiment")

'2'

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: beef91ed7c494e2cb33e68f72df1ff82, rmse: 11.1673
run id: 831f70d4497044b18f6b34446eff30e6, rmse: 11.1673
run id: e005f8bf5c424b42af43d342e8df9f28, rmse: 12.1432
run id: cd8e2b1fbe1447ae97a8729dd363caaf, rmse: 12.1432
run id: a7d056a1b4dd4f5ca688179a2f1b6fd2, rmse: 12.1432


In [18]:
import mlflow

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "2dea7ac6c1fb449ab7dafd18fda3d48c"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748116648130, current_stage='None', description=None, last_updated_timestamp=1748116648130, name='nyc-taxi-regressor', run_id='2dea7ac6c1fb449ab7dafd18fda3d48c', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dea7ac6c1fb449ab7dafd18fda3d48c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/tmp/ipykernel_5540/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [21]:
results = client.search_registered_models(order_by=["name ASC"])

for res in results:
    for lv in res.latest_versions:
        print(f"version: {version.version}, stage: {version.current_stage}")       

version: 3, stage: None


In [24]:
model_name = "nyc-taxi-regressor"
model_version = 3
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_5540/1665145830.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748116648130, current_stage='Staging', description=None, last_updated_timestamp=1748118140652, name='nyc-taxi-regressor', run_id='2dea7ac6c1fb449ab7dafd18fda3d48c', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dea7ac6c1fb449ab7dafd18fda3d48c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [26]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1748116648130, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2025-05-24', last_updated_timestamp=1748118320831, name='nyc-taxi-regressor', run_id='2dea7ac6c1fb449ab7dafd18fda3d48c', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dea7ac6c1fb449ab7dafd18fda3d48c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [20]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [7]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [12]:
run_id = "693939ca496d421aa0252288d5ff1110"

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [13]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)



In [14]:
X_test = preprocess(df, dv)

In [15]:
target = "duration"
y_test = df[target].values

In [23]:
model_name = "nyc-taxi-regressor"
%time test_model(name=model_name, stage='None', X_test=X_test, y_test=y_test)

CPU times: user 9.35 s, sys: 258 ms, total: 9.61 s
Wall time: 3.33 s


{'rmse': 6.519651442307065}

In [24]:
model_name = "nyc-taxi-regressor"
%time test_model(name=model_name, stage='Staging', X_test=X_test, y_test=y_test)

CPU times: user 9.12 s, sys: 31.9 ms, total: 9.15 s
Wall time: 2.58 s


{'rmse': 6.519651442307065}